In [1]:
import cv2
import numpy as np

In [2]:

import cv2
from darkflow.net.build import TFNet
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'svg'
%matplotlib inline 

C:\Users\Ilya\Kyrsach\Kyrsovaya\darkflow-master


c:\anaconda\envs\maskrcnn\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
optinos = {
    'model': 'cfg/yolo.cfg',
    'load': 'bin/yolo.weights',
    'threshold': 0.3,
    'gpu': 1.0
}

In [4]:
tfnet = TFNet(optinos)

Parsing ./cfg/yolo.cfg
Parsing cfg/yolo.cfg
Loading bin/yolo.weights ...
Successfully identified 203934260 bytes
Finished in 0.014515399932861328s
Model has a coco model name, loading coco labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 608, 608, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 76, 76, 128)
 Load  |  Yep

In [12]:
%cd Kyrsovaya/

C:\Users\Ilya\Kyrsach\Kyrsovaya


In [13]:
%ls

 Том в устройстве C не имеет метки.
 Серийный номер тома: F4E6-4E15

 Содержимое папки C:\Users\Ilya\Kyrsach\Kyrsovaya

28.04.2018  14:58    <DIR>          .
28.04.2018  14:58    <DIR>          ..
09.04.2018  20:56    <DIR>          .idea
28.04.2018  14:51    <DIR>          .ipynb_checkpoints
24.04.2018  20:28    <DIR>          __pycache__
27.04.2018  22:39    <DIR>          darkflow-master
19.04.2018  11:24    <DIR>          data
27.04.2018  23:33             4 034 MainProject.ipynb
28.04.2018  14:50           102 174 PeopleCountingLesson.ipynb
19.04.2018  14:23             2 202 Person.py
09.04.2018  20:56               908 Project.py
22.04.2018  17:45    <DIR>          re3-tensorflow
28.04.2018  14:58             9 331 Test.ipynb
27.04.2018  21:34    <DIR>          utils
               5 файлов        118 649 байт
               9 папок  56 923 463 680 байт свободно


In [26]:
#################################################
#                                               #
#        Рисуем рамку движ. объекта             #
#                                               #
#                                               #
#################################################

# не могу брать видео, где камера не зафиксирована 

cap = cv2.VideoCapture('./data/peopleCounter.avi')
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=True)
kernelOp = np.ones((3,3), np.uint8)
kernelCl = np.ones((11,11), np.uint8)
areaTH = 800

while(cap.isOpened()):
    ok, frame = cap.read()
    fgmask = fgbg.apply(frame)
    try:
        ret, imBin = cv2.threshold(fgmask, 200, 255, cv2.THRESH_BINARY)
        mask = cv2.morphologyEx(imBin, cv2.MORPH_OPEN, kernelOp)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernelCl)
    except:
        print('EOF')
        break
    cv2.imshow('ss', imBin)
    _, contours0, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL,\
                                               cv2.CHAIN_APPROX_NONE)
    for cnt in contours0:
        area = cv2.contourArea(cnt)
        if area > areaTH:
            #################
            #   TRACKING    #
            #################            
            M = cv2.moments(cnt)
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            x,y,w,h = cv2.boundingRect(cnt)
            cv2.circle(frame,(cx,cy), 5, (0,0,255), -1)            
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
            cv2.drawContours(frame, cnt, -1, (0,255,0), 1, 8)
    cv2.imshow('Contours', frame)

    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
        
cap.release()
cv2.destroyAllWindows()
    

EOF
